In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("Exercise_1_lab8_DE2")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")
# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()
# Load data from BigQuery.

conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

gsc_file_path = 'gs://data_de2/transfers.csv'  #  use your gcp bucket name. Also upload sales.csv first

df_transfers = spark.read.format("csv").option("header", "true").option("delimiter", ",") \
       .load(gsc_file_path)
df_transfers.printSchema()
df_transfers.show(5)

df_players = spark.read \
  .format("bigquery") \
  .load("de2024-436414.transfers_kaggle.players")    # project_id.datatset.tablename. Use your project id
df_players.printSchema()
df_players.show(4)



root
 |-- player_id: string (nullable = true)
 |-- transfer_date: string (nullable = true)
 |-- transfer_season: string (nullable = true)
 |-- from_club_id: string (nullable = true)
 |-- to_club_id: string (nullable = true)
 |-- from_club_name: string (nullable = true)
 |-- to_club_name: string (nullable = true)
 |-- transfer_fee: string (nullable = true)
 |-- market_value_in_eur: string (nullable = true)
 |-- player_name: string (nullable = true)

+---------+-------------+---------------+------------+----------+--------------+---------------+------------+-------------------+---------------+
|player_id|transfer_date|transfer_season|from_club_id|to_club_id|from_club_name|   to_club_name|transfer_fee|market_value_in_eur|    player_name|
+---------+-------------+---------------+------------+----------+--------------+---------------+------------+-------------------+---------------+
|   195778|   2026-06-30|          25/26|          79|        27| VfB Stuttgart|  Bayern Munich|       0.000|

In [12]:
df_players.columns

['player_id',
 'first_name',
 'last_name',
 'name',
 'last_season',
 'current_club_id',
 'player_code',
 'country_of_birth',
 'city_of_birth',
 'country_of_citizenship',
 'date_of_birth',
 'sub_position',
 'position',
 'foot',
 'height_in_cm',
 'contract_expiration_date',
 'agent_name',
 'image_url',
 'url',
 'current_club_domestic_competition_id',
 'current_club_name',
 'market_value_in_eur',
 'highest_market_value_in_eur']

In [20]:
df_transfers.columns

['player_id',
 'transfer_date',
 'transfer_season',
 'from_club_id',
 'to_club_id',
 'from_club_name',
 'to_club_name',
 'transfer_fee',
 'market_value_in_eur',
 'player_name']

**What are the top 3 deals of each agent?**

The 'goodness' of a deal will be defined by substracting the market_value from the transfer fee. The higher this number, the more the club 'overpaid' for the player according to the data from transfermarkt


In [19]:
# Again, we only select the columns from df_players that are not in df_transfers and insightful
df_players_new = df_players.select("player_id", "player_code", "country_of_birth",  
                                   "sub_position", "position",  "agent_name" ) 

# We also select only relevant columns from the df_transfers 
df_transfers_new = df_transfers.select("player_id", "player_name", "transfer_date", "from_club_name", "to_club_name", "transfer_fee", "market_value_in_eur")


df_players_new.show(15)

+---------+--------------------+----------------+------------+--------+----------------+
|player_id|         player_code|country_of_birth|sub_position|position|      agent_name|
+---------+--------------------+----------------+------------+--------+----------------+
|    56607|         anco-jansen|     Netherlands|        NULL| Missing|            NULL|
|   231165|      fotis-doumanis|          Greece|        NULL| Missing|            NULL|
|   727111|anastasios-papado...|            NULL|        NULL| Missing|            NULL|
|   564561|      mathurin-sakho|            NULL|        NULL| Missing|       ProSkills|
|   158704|   jon-ander-garrido|           Spain|        NULL| Missing|    JV SPORTS SL|
|   457129|      ben-hinchliffe|         England|        NULL| Missing|            NULL|
|   584378|        lucas-geurde|            NULL|        NULL| Missing|            NULL|
|  1160121|      wassim-zaatout|            NULL|        NULL| Missing|            NULL|
|   451848| geoffrey-

In the table above, we see that a lot of agents are missing. These players likely do not have an agent. Since the agent is what we are interested in, we will later remove these players from the merged dataset.

In [8]:
players_merged_clean.printSchema()

root
 |-- player_id: string (nullable = true)
 |-- player_name: string (nullable = true)
 |-- transfer_date: string (nullable = true)
 |-- from_club_name: string (nullable = true)
 |-- to_club_name: string (nullable = true)
 |-- transfer_fee: string (nullable = true)
 |-- market_value_in_eur: string (nullable = true)
 |-- player_id: long (nullable = true)
 |-- player_code: string (nullable = true)
 |-- country_of_birth: string (nullable = true)
 |-- sub_position: string (nullable = true)
 |-- position: string (nullable = true)
 |-- agent_name: string (nullable = true)



We again see that we need to change some types, which we will do in the next cell. We will also merge the datasets here and drop rows that have null values for either transfer fee, market value, or agent (player without an agent). 

In [11]:
from pyspark.sql.functions import *
from pyspark.sql import Window

# Join the player and transfer datasets
join_expression = df_players_new["player_id"] == df_transfers_new["player_id"]
players_merged = df_transfers_new.join(df_players_new, join_expression, "left").drop("rid")

# Filter out rows with null values in key columns
players_merged_clean = players_merged.filter(col("transfer_fee").isNotNull())
players_merged_clean = players_merged_clean.filter(col("market_value_in_eur").isNotNull())

# Convert transfer_date to date type (without timestamp)
players_merged_clean = players_merged_clean.withColumn("transfer_date", to_date(col("transfer_date"), "yyyy-MM-dd"))

# Convert market_value_in_eur and transfer_fee to integer type
players_merged_clean = players_merged_clean.withColumn("market_value_in_eur", col("market_value_in_eur").cast("int")) \
                                           .withColumn("transfer_fee", col("transfer_fee").cast("int"))

players_merged_clean = players_merged_clean.na.drop("all", subset=["agent_name"])


# Verify the schema after changes
players_merged_clean.printSchema()

root
 |-- player_id: string (nullable = true)
 |-- player_name: string (nullable = true)
 |-- transfer_date: date (nullable = true)
 |-- from_club_name: string (nullable = true)
 |-- to_club_name: string (nullable = true)
 |-- transfer_fee: integer (nullable = true)
 |-- market_value_in_eur: integer (nullable = true)
 |-- player_id: long (nullable = true)
 |-- player_code: string (nullable = true)
 |-- country_of_birth: string (nullable = true)
 |-- sub_position: string (nullable = true)
 |-- position: string (nullable = true)
 |-- agent_name: string (nullable = true)



Here we continue with getting the dataset we want to fulfill our goal. 

In [14]:
# Calculate the "deal" as the difference between transfer_fee and market_value
players_merged_clean = players_merged_clean.withColumn("deal", col("transfer_fee") - col("market_value_in_eur"))

# Define a window specification to rank deals for each agent
window_spec = Window.partitionBy("agent_name").orderBy(col("deal").desc())

# Add row numbers to identify the top 3 deals for each agent
ranked_df = players_merged_clean.withColumn("deal_ranked", row_number().over(window_spec)).filter(col("deal_ranked") <= 3)

# Calculate the average deal and the number of deals for each agent
average_df = df_agents.groupBy("agent_name").agg(
    round(avg("deal"), 1).alias("average_deal_agent"),  # Average deal
    count("deal").alias("nr_of_deals_agent")  # Total number of deals for each agent
)

# Join the top deals with the average stats
result_df = ranked_df.join(average_df, on="agent_name", how="inner")

# Select and display the final table
top_three_deals_table = result_df.select(
    "agent_name", 
    "deal_ranked", 
    "player_name", 
    "from_club_name", 
    "to_club_name", 
    "deal", 
    "transfer_fee", 
    "market_value_in_eur", 
    "nr_of_deals_agent", 
    "average_deal_agent"
).orderBy(col("agent_name"), col("deal_ranked").asc())

top_three_deals_table.show(20)


+-------------------+-----------+--------------------+---------------+--------------+--------+------------+-------------------+-----------------+------------------+
|         agent_name|deal_ranked|         player_name| from_club_name|  to_club_name|    deal|transfer_fee|market_value_in_eur|nr_of_deals_agent|average_deal_agent|
+-------------------+-----------+--------------------+---------------+--------------+--------+------------+-------------------+-----------------+------------------+
|         EB SPORTS |          1|         Aliou Baldé| Lausanne-Sport|      OGC Nice| 1700000|     2500000|             800000|               24|         -668458.3|
|         EB SPORTS |          2|          Mame Thiam|Virtus Lanciano|      Juventus|  700000|     1400000|             700000|               24|         -668458.3|
|         EB SPORTS |          3|         Aliou Baldé|  Feyenoord U21|Lausanne-Sport|  600000|     1000000|             400000|               24|         -668458.3|
|         

In [20]:
# Use the Cloud Storage bucket for temporary BigQuery export data the connector uses.
bucket = "temp_de2"
spark.conf.set('temporaryGcsBucket', bucket)
# Saving the data to BigQuery
top_three_deals_table.write.format('bigquery') \
  .option('table', 'de2024-436414.transfers_kaggle.best3_agentdeals') \
  .mode("overwrite") \
  .save()

In [42]:
# Stop the spark context
spark.stop()